In [1]:
import numpy as np 
import os
import mne
import pandas as pd
from itertools import compress
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import re
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics


    

In [ ]:
# Define the data path
data_path = r'C:\Users\takoy\Documents\mind-tbi-fnirs\neurotech_fnir_mind\data'

# Define stimulation condition: 'real' for 20 min stimulation or 'sham' for 30 s stimulation.
# In both cases, a fade-off period of 30 seconds is applied at the end.

#yeah I don't think the files follow the study
#files are about 7.5 mins each
condition = 'real'  # Change to 'sham' if needed
fade_duration_sec = 30  # Fade-off duration (seconds)

if condition == 'real':
    stim_on_duration_sec = 20 * 60  # 20 minutes in seconds
elif condition == 'sham':
    stim_on_duration_sec = 30       # 30 seconds
else:
    raise ValueError("Condition must be 'real' or 'sham'.")

# Total stimulation period includes the fade-off period
#total_stim_duration_sec = stim_on_duration_sec + fade_duration_sec

# Define window length for segmentation and averaging.
#window_length_sec = total_stim_duration_sec  

for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    if not file.endswith(".snirf"):
        continue
    print(file_path)

    # Read the SNIRF file and load data
    raw_intensity = mne.io.read_raw_snirf(file_path, verbose=True)
    raw_intensity.load_data()

    total_stim_duration_sec = raw_intensity.n_times / raw_intensity.info['sfreq']

    # Select fNIRS channels and filter out channels with too short source-detector distances
    picks = mne.pick_types(raw_intensity.info, meg=False, fnirs=True)
    dists = mne.preprocessing.nirs.source_detector_distances(
        raw_intensity.info, picks=picks
    )
    raw_intensity.pick(picks[dists > 0.01])
    
    # Convert intensity data to optical density
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    # Compute scalp coupling index and mark channels with low coupling as bad
    sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
    raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))

    # Convert optical density to haemoglobin concentrations using the Beer-Lambert Law
    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)
    raw_haemo_unfiltered = raw_haemo.copy()

    # Filter the haemoglobin data
    raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)

    # Create a binary stimulus channel based on the actual stimulation period
    stim_ch = np.zeros((1, raw_haemo.n_times))
    sfreq = raw_haemo.info['sfreq']

    # Define stimulation start time (in seconds); adjust if there is a baseline period
    stim_start_sec = 0  
    stim_end_sec = stim_start_sec + total_stim_duration_sec

    # Convert stimulation start and end times to sample indices
    stim_start_index = int(stim_start_sec * sfreq)
    stim_end_index = int(stim_end_sec * sfreq)

    # Mark the stimulation period (including fade-off) with 1's; elsewhere remains 0
    stim_ch[0, stim_start_index:stim_end_index] = 1

    # Create an MNE RawArray for the stimulus channel and add it to the haemoglobin data
    stim_info = mne.create_info(['stim'], sfreq, ch_types=['stim'])
    stim_raw = mne.io.RawArray(stim_ch, stim_info)
    raw_haemo.add_channels([stim_raw], force_update_info=True)

    print("Data shape after adding stim channel:", raw_haemo.get_data().shape)

    # We use only the portion where the stim channel is 1.
    #window_samples = int(total_stim_duration_sec * sfreq)
    #stim_data_length = stim_end_index - stim_start_index
    #n_windows = stim_data_length // window_samples
    #if n_windows == 0:
        #print("Not enough data for even one window in file:", file)
        #continue

    # Initialize lists to collect segments for HbO and HbR channels
    segments_hbo = []
    segments_hbr = []
    
    # Extract non-overlapping segments from the stimulation period
    for i in range(n_windows):
        seg_start = stim_start_index + i * window_samples
        seg_end = seg_start + window_samples
        seg = raw_haemo.get_data(picks=["hbo", "hbr"], start=seg_start, stop=seg_end)
        # seg shape: (2, window_samples) where index 0 is HbO and 1 is HbR
        segments_hbo.append(seg[0, :])
        segments_hbr.append(seg[1, :])
    
    # Convert list of segments to numpy arrays with shape (n_windows, window_samples)
    segments_hbo = np.array(segments_hbo)
    segments_hbr = np.array(segments_hbr)
    
    # Compute the average waveform (across windows) for each channel
    avg_hbo = np.mean(segments_hbo, axis=0)
    avg_hbr = np.mean(segments_hbr, axis=0)
    
    # Create a time vector (in seconds) for one window
    #time_vector = np.linspace(0, window_length_sec, num=window_samples, endpoint=False)
    time_vector = np.linspace(0, total_stim_duration_sec, num=1, endpoint=False)

    print(len(time_vector), len(avg_hbo), len(avg_hbr))

    # Create a DataFrame that will be saved as CSV.
    # The DataFrame contains the time (relative to window start) and the averaged HbO and HbR values.
    df = pd.DataFrame({
        "Time (s)": time_vector,
        "HbO_avg": avg_hbo,
        "HbR_avg": avg_hbr
    })
    
    # Save the DataFrame to a CSV file in the same folder
    out_csv = file.replace(".snirf", "_averaged.csv")
    out_csv_path = os.path.join(data_path, out_csv)
    df.to_csv(out_csv_path, index=False)
    print("Saved averaged data CSV to:", out_csv_path)
    

C:\Users\takoy\Documents\mind-tbi-fnirs\neurotech_fnir_mind\data\rest15.snirf
Loading C:\Users\takoy\Documents\mind-tbi-fnirs\neurotech_fnir_mind\data\rest15.snirf
Reading 0 ... 22949  =      0.000 ...   458.980 secs...


C:\Users\takoy\AppData\Local\Temp\ipykernel_21808\3383588010.py:32: RuntimeWarning: The data only contains 2D location information for the optode positions. It is highly recommended that data is used which contains 3D location information for the optode positions. With only 2D locations it can not be guaranteed that MNE functions will behave correctly and produce accurate results. If it is not possible to include 3D positions in your data, please consider using the set_montage() function.
  raw_intensity = mne.io.read_raw_snirf(file_path, verbose=True)
C:\Users\takoy\AppData\Local\Temp\ipykernel_21808\3383588010.py:45: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 8251 samples (165.020 s)

Creating RawArray with float64 data, n_channels=1, n_times=22950
    Range : 0 ... 22949 =      0.000 ...   458.980 secs
Ready.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s


Data shape after adding stim channel: (73, 22950)
61500 22950 22950


ValueError: All arrays must be of the same length

In [7]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = None

#X_train, X_test, y_train, y_test = train_test_split(, , test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions:
y_pred = model.predict(X_test)

#Get data from fNIRs (Elan)

# calculating the mean values.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cm = metrics.confusion_matrix(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
plt.scatter(X_test, y_test, color='blue')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()


TypeError: cannot unpack non-iterable NoneType object